In [1]:
import functools
import json
import logging
import os
import re
import sys
import warnings
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Union
from dataset import BengaliDataset
import datasets
import evaluate
import numpy as np
import torch
from datasets import DatasetDict, load_dataset
import pandas as pd
import transformers
from transformers import (
    AutoConfig,
    AutoFeatureExtractor,
    AutoModelForCTC,
    AutoProcessor,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    Wav2Vec2Processor,
    set_seed,
)

from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor
) 
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

2023-09-06 10:39:07.736525: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 10:39:08.465696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-09-06 10:39:09,563] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
df_ = pd.read_parquet("/home/ubuntu/bengali/data/train_duration.parquet")

In [3]:
len(df_)

963636

In [10]:
text_list = []
file_path = "/home/ubuntu/Raijin/bn_dedup.txt"
with open(file_path, "r") as file_input:
    for line in file_input:
#         print(line)
        text_list.append(line)

In [11]:
len(text_list)

7679439

In [12]:
scraped_df = pd.read_csv("/home/ubuntu/Raijin/Bengali_News_1.csv")
len(scraped_df)

309711

In [13]:
scraped_df.columns

Index(['Headlines', 'Highlights'], dtype='object')

In [14]:
scraped_list = []
for x in range(len(scraped_df)):
    head = scraped_df["Headlines"][x]
    high = scraped_df["Highlights"][x]
    scraped_list.append(f"{head} {high}")

In [15]:
scraped_df["Headlines"][0]

'এবার ক্যাম্পাসে আসছে ইউজিসি’\u200cর বিশেষ টিম, যাদবপুর বিশ্ববিদ্যালয়ের রিপোর্ট কেমন?'

In [16]:
df_["sentence"][0]

'ও বলেছে আপনার ঠিকানা!'

In [17]:
base_list = []
for x in range(len(df_)):
    base_list.append(df_["sentence"][x])
    
len(base_list), base_list[0:2]

(963636, ['ও বলেছে আপনার ঠিকানা!', 'কোন মহান রাষ্ট্রের নাগরিক হতে চাও?'])

In [18]:
combined_list = base_list + scraped_list + text_list
len(combined_list)

8952786

In [19]:
len(base_list), len(scraped_list), len(text_list)

(963636, 309711, 7679439)

In [20]:
from bnunicodenormalizer import Normalizer 

In [21]:
bnorm=Normalizer()
# normalize
word = 'াটোবাকোওবলেছেআপনারঠিকানা!'
result=bnorm(word)
result

{'normalized': 'টোবাকোওবলেছেআপনারঠিকানা!',
 'given': 'াটোবাকোওবলেছেআপনারঠিকানা!',
 'ops': [{'operation': 'InvalidUnicode',
   'before': 'াটোবাকোওবলেছেআপনারঠিকানা!',
   'after': 'টোবাকোওবলেছেআপনারঠিকানা!'}]}

In [22]:
result["ops"][0]

{'operation': 'InvalidUnicode',
 'before': 'াটোবাকোওবলেছেআপনারঠিকানা!',
 'after': 'টোবাকোওবলেছেআপনারঠিকানা!'}

In [ ]:
# normalized_list = []
# for x in range(len(combined_list)):
    
#     words = combined_list[x].split(" ")
#     complete_sentence = []
    
#     for word in words:
#         norm = bnorm(word)
        
#         if len(norm["ops"]) != 0:
#             output = norm["ops"][0]["after"]
#         else:
#             output = norm["normalized"]
            
#         complete_sentence.append(output)
        
#     try:    
#         full_string = " ".join(complete_sentence)
#     except Exception as e:
#         full_string = " "
#     finally:
#         normalized_list.append(full_string)

In [23]:
len(combined_list)

8952786